# Get Origin Block Groups from Patterns

White Pass Ski area, when you ride up on the lift it is common to discover people next to you hail from Yakima, the Tri-Cities, Portland-Vancouver, Olympia and Tacoma. White Pass is well known for being a very low-key, friendly and extremely unpretentious ski area. With Mt. Hood near Portland and Stephens Pass and Crystal near Seattle getting increasingly crowded, there is a belief more people are beginning to think the drive to White Pass is worth it. Talking to people on the lift, though, is little more than antecdoctal. 

Where are people _really_ coming from, and are these patterns changing? We can investigate these trends using Safegraph Patterns data.

In [1]:
# this url will be used later, but it's here at the top for easily finding
# the url to block groups on ArcGIS Online may change, so you may need to change this later

bg_url = 'https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Block_Groups/FeatureServer/0/'

In [44]:
import importlib
import json
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, FeatureLayer
import pandas as pd
from modeling import Country

ModuleNotFoundError: No module named 'modeling'

In [40]:
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'

dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

Using the previously downloaded patterns data for White Pass, the first step is simply loading the data into a Pandas DataFrame. This DataFrame contains quite a few columns encoded as dictionaries, which can be expanded for more data detail.

In [4]:
wp_pth = dir_raw/'patterns_white_pass.csv'

In [5]:
wp_df = pd.read_csv(wp_pth, index_col=0)

wp_df.head()

,placekey,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,...,naics_code,latitude,longitude,iso_country_code,phone_number,open_hours,opened_on,closed_on,tracking_opened_since,tracking_closed_since
169339,zzy-222@5xd-7jh-f75,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-10-01T00:00:00-07:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233622,zzw-222@5xd-7jh-ffz,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2020-11-01T00:00:00-07:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94681,NaN,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2018-01-01T00:00:00-08:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128545,NaN,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2018-02-01T00:00:00-08:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119562,NaN,sg:af471021a929414cbf69854e6f8f1b0c,White Pass Ski Area,48935 U.s. 12,Naches,WA,98937,NaN,NaN,2018-03-01T00:00:00-08:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One of these columns `visitor_home_cbgs` is a dictionary of Census Block Groups with the count of unique devices originating in the block group visiting the point of interest, in this case White Pass Ski Area, in each of the months. Since this column is read in as a string from the CSV, we are going to convert it to a dictionary using `json.loads`.

In [6]:
wp_df.visitor_home_cbgs = wp_df.visitor_home_cbgs.apply(lambda val: json.loads(val))

wp_df.visitor_home_cbgs

169339    {'530730011001': 4, '530770028012': 4, '530350...
233622    {'530530724102': 7, '530530715031': 6, '530670...
94681     {'530670117102': 10, '530770034001': 7, '53015...
128545    {'530530724051': 10, '530530724063': 7, '53067...
119562    {'530499505002': 8, '530770032001': 7, '530770...
143458    {'530770030022': 8, '530770028012': 5, '530670...
168211    {'530670127301': 5, '530419720003': 4, '410459...
228611    {'530770028012': 11, '530419720003': 10, '5306...
178995    {'530419720003': 11, '530670117103': 9, '53005...
158825    {'530530712062': 9, '530770028012': 8, '530670...
137495    {'530419720003': 11, '530770008001': 7, '53077...
187308    {'530530723082': 9, '410510101003': 6, '530770...
129252    {'530330320112': 5, '530419717002': 4, '530770...
167450    {'530770034001': 9, '530530724052': 9, '530530...
140580    {'530770028022': 12, '530670120001': 9, '53033...
159573    {'530419720003': 12, '530530728002': 7, '53077...
110570    {'530459613001': 11, '53077001

The ski season generally runs November through March. Although we have data for the entire year, we are only going to focus on the months comprising the ski season. To do this, we need to extract the year and month explicitly from a datetime field. From there, we can filter to just the months we are interested in.

In [7]:
mnths = [11, 12, 1, 2, 3, 4]

wp_df['year'] = pd.to_datetime(wp_df.date_range_start).apply(lambda dt: dt.year)
wp_df['month'] = pd.to_datetime(wp_df.date_range_start).apply(lambda dt: dt.month)

dict_df = wp_df[wp_df.month.isin(mnths)][['year', 'month', 'visitor_home_cbgs']].sort_values(['year', 'month'])

Since, at least in this case, we are interested in trends for each ski season as a whole, we are also going to create a label we can later use to bin the data.

In [8]:
dict_df['season'] = dict_df[['year', 'month']].apply(lambda r: f'{r.year-1}_{r.year}' if r.month < 6 else f'{r.year}_{r.year+1}', axis=1)

dict_df.head()

,year,month,visitor_home_cbgs,season
94681,2018,1,"{'530670117102': 10, '530770034001': 7, '53015...",2017_2018
333526,2018,1,"{'530150020021': 8, '530530731161': 8, '530050...",2017_2018
128545,2018,2,"{'530530724051': 10, '530530724063': 7, '53067...",2017_2018
157487,2018,2,"{'530770028023': 12, '530330001003': 8, '53067...",2017_2018
119562,2018,3,"{'530499505002': 8, '530770032001': 7, '530770...",2017_2018


Next, although a trival step, we reduce the dataframe down to just the columns we need, the binning season and the dictionary of home block groups and device counts.

In [9]:
dict_df = dict_df[['season', 'visitor_home_cbgs']]

dict_df.head()

,season,visitor_home_cbgs
94681,2017_2018,"{'530670117102': 10, '530770034001': 7, '53015..."
333526,2017_2018,"{'530150020021': 8, '530530731161': 8, '530050..."
128545,2017_2018,"{'530530724051': 10, '530530724063': 7, '53067..."
157487,2017_2018,"{'530770028023': 12, '530330001003': 8, '53067..."
119562,2017_2018,"{'530499505002': 8, '530770032001': 7, '530770..."


Next, we explode out the all the values from the dictionary to be columns in the dataframe.

In [10]:
season_df = dict_df[['season']].join(dict_df.visitor_home_cbgs.apply(pd.Series))

season_df.head()

,season,530670117102,530770034001,530150009003,530770008001,530050108033,530459613002,530530728001,350579632021,530419720003,...,530530715031,530670115003,410279504002,530350928023,530210206011,530530703093,530530604003,530530623002,530459605003,530330082002
94681,2017_2018,10.0,7.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
333526,2017_2018,4.0,4.0,NaN,NaN,7.0,4.0,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128545,2017_2018,4.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157487,2017_2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119562,2017_2018,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we can simply use `groupby` and `max` to get the maximum number of devices per month originating from each block group for each season. Also, while we are at it, we can fill in any missing values with zeros.

In [11]:
season_df = season_df.groupby('season').max().fillna(0)

season_df.head()

,530670117102,530770034001,530150009003,530770008001,530050108033,530459613002,530530728001,350579632021,530419720003,530530606001,...,530530715031,530670115003,410279504002,530350928023,530210206011,530530703093,530530604003,530530623002,530459605003,530330082002
season,,,,,,,,,,,,,,,,,,,,,
2017_2018,10.0,7.0,6.0,6.0,7.0,6.0,6.0,6.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018_2019,4.0,5.0,0.0,9.0,5.0,4.0,7.0,0.0,11.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019_2020,7.0,9.0,0.0,9.0,6.0,0.0,4.0,0.0,12.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020_2021,0.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,6.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


With all this organized, now we can organize the data by the block group id using `transpose`.

In [12]:
season_bg_df = season_df.transpose()

season_bg_df

season,2017_2018,2018_2019,2019_2020,2020_2021
530670117102,10.0,4.0,7.0,0.0
530770034001,7.0,5.0,9.0,4.0
530150009003,6.0,0.0,0.0,0.0
530770008001,6.0,9.0,9.0,4.0
530050108033,7.0,5.0,6.0,4.0
...,...,...,...,...
530530703093,0.0,0.0,0.0,4.0
530530604003,0.0,0.0,0.0,4.0
530530623002,0.0,0.0,0.0,4.0
530459605003,0.0,0.0,0.0,4.0


In [13]:
dlta_df = (season_df - season_df.shift()).transpose()
dlta_df.drop(columns=dlta_df.columns[0], inplace=True)
dlta_df.columns = [f'{c}_delta' for c in dlta_df.columns]

dlta_df

,2018_2019_delta,2019_2020_delta,2020_2021_delta
530670117102,-6.0,3.0,-7.0
530770034001,-2.0,4.0,-5.0
530150009003,-6.0,0.0,0.0
530770008001,3.0,0.0,-5.0
530050108033,-2.0,1.0,-2.0
...,...,...,...
530530703093,0.0,0.0,4.0
530530604003,0.0,0.0,4.0
530530623002,0.0,0.0,4.0
530459605003,0.0,0.0,4.0


In [35]:
season_delta_df = season_bg_df.join(dlta_df)
season_delta_df = season_delta_df[season_delta_df.columns.sort_values()]
season_delta_df.columns = [f'season_{c}' for c in season_delta_df.columns]

season_delta_df

,season_2017_2018,season_2018_2019,season_2018_2019_delta,season_2019_2020,season_2019_2020_delta,season_2020_2021,season_2020_2021_delta
530670117102,10.0,4.0,-6.0,7.0,3.0,0.0,-7.0
530770034001,7.0,5.0,-2.0,9.0,4.0,4.0,-5.0
530150009003,6.0,0.0,-6.0,0.0,0.0,0.0,0.0
530770008001,6.0,9.0,3.0,9.0,0.0,4.0,-5.0
530050108033,7.0,5.0,-2.0,6.0,1.0,4.0,-2.0
...,...,...,...,...,...,...,...
530530703093,0.0,0.0,0.0,0.0,0.0,4.0,4.0
530530604003,0.0,0.0,0.0,0.0,0.0,4.0,4.0
530530623002,0.0,0.0,0.0,0.0,0.0,4.0,4.0
530459605003,0.0,0.0,0.0,0.0,0.0,4.0,4.0


In [36]:
season_delta_df.to_csv(dir_int/'block_group_raw.csv')

## Get Geometry from Block Groups on ArcGIS Online

In [16]:
bg_lyr = FeatureLayer(bg_url)

bg_lyr

<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Block_Groups/FeatureServer/0">

In [37]:
bg_id_lst = dlta_df.index
bg_id_str_lst = [f"'{bgid}'" for bgid in bg_id_lst]
bg_id_str = ','.join(bg_id_str_lst)

where_str = f'FIPS IN ({bg_id_str})'

In [18]:
bg_shp_df = bg_lyr.query(where_str, out_fields=['FIPS']).sdf

bg_shp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID_1  1148 non-null   int64   
 1   FIPS        1148 non-null   object  
 2   SHAPE       1148 non-null   geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 27.0+ KB


In [38]:
bg_df = bg_shp_df.join(season_delta_df, on='FIPS').drop(columns='OBJECTID_1')
bg_df.spatial.set_geometry('SHAPE')

bg_df.head()

,FIPS,SHAPE,season_2017_2018,season_2018_2019,season_2018_2019_delta,season_2019_2020,season_2019_2020_delta,season_2020_2021,season_2020_2021_delta
0,483396941011,"{""rings"": [[[-95.4701659369084, 30.42173599202...",0.0,4.0,4.0,0.0,-4.0,0.0,0.0
1,481677212021,"{""rings"": [[[-95.0010579532619, 29.50889505995...",0.0,0.0,0.0,4.0,4.0,0.0,-4.0
2,483396920022,"{""rings"": [[[-95.3564050615443, 30.07689500130...",0.0,0.0,0.0,4.0,4.0,0.0,-4.0
3,150030099022,"{""rings"": [[[-158.10524106952, 21.579959059762...",0.0,4.0,4.0,0.0,-4.0,0.0,0.0
4,150030101003,"{""rings"": [[[-157.949919027641, 21.69243197015...",0.0,4.0,4.0,0.0,-4.0,0.0,0.0


In [39]:
bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   FIPS                    1148 non-null   object  
 1   SHAPE                   1148 non-null   geometry
 2   season_2017_2018        1148 non-null   float64 
 3   season_2018_2019        1148 non-null   float64 
 4   season_2018_2019_delta  1148 non-null   float64 
 5   season_2019_2020        1148 non-null   float64 
 6   season_2019_2020_delta  1148 non-null   float64 
 7   season_2020_2021        1148 non-null   float64 
 8   season_2020_2021_delta  1148 non-null   float64 
dtypes: float64(7), geometry(1), object(1)
memory usage: 80.8+ KB


In [41]:
bg_df.spatial.to_featureclass(gdb_raw/'block_group_patterns')

'D:\\projects\\white-pass-feature-selection\\data\\raw\\raw.gdb\\block_group_patterns'

In [42]:
bg_df.SHAPE = bg_df.SHAPE.apply(lambda geom: json.dumps(geom.JSON))

bg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fips                    1148 non-null   object 
 1   SHAPE                   1148 non-null   object 
 2   season_2017_2018        1148 non-null   float64
 3   season_2018_2019        1148 non-null   float64
 4   season_2018_2019_delta  1148 non-null   float64
 5   season_2019_2020        1148 non-null   float64
 6   season_2019_2020_delta  1148 non-null   float64
 7   season_2020_2021        1148 non-null   float64
 8   season_2020_2021_delta  1148 non-null   float64
dtypes: float64(7), object(2)
memory usage: 80.8+ KB


In [43]:
bg_df.to_csv(dir_int/'block_group_patterns.csv')